# Table of Contents
 <p>

In [13]:
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from traitlets import Unicode, Bool, validate, TraitError
import ipywidgets as widgets
from ipywidgets import DOMWidget, register
from IPython.display import display, HTML, JSON, Code
from vega import Vega
from vega3 import Vega as Vega3
import requests
import json
import re

%reload_ext version_information
%version_information vega, vega3, IPython, ipywidgets, re

Software versions
Python 3.7.4 64bit [Clang 10.0.1 (clang-1001.0.46.4)]
IPython 7.7.0
OS Darwin 18.6.0 x86_64 i386 64bit
vega 0.5.0
vega3 0.5.0
IPython 7.7.0
ipywidgets 7.5.1
re 2.2.1
Fri Aug 02 10:10:23 2019 CEST

In [ ]:
AQvegaTheme="""
{
  "background":"#fff",
  "render": {"retina": true},
  "axis": {
    "layer": "back",
    "ticks": 5,
    "axisColor": "transparent",
    "axisWidth": 1,
    "gridColor": "#eeeeee",
    "gridOpacity": 1,
    "tickColor": "#eeeeee",
    "tickLabelColor": "#758290",
    "tickWidth": 0,
    "tickSize": 10,
    "tickLabelFontSize": 12,
    "tickLabelFont": "\"Roboto\"",
    "tickOffset": 0,
    "titleFont": "\"Roboto\"",
    "titleFontSize": 12,
    "titleFontWeight": "regular",
    "titleColor": "#758290",
    "titleOffset": "auto",
    "titleOffsetAutoMin": 0,
    "titleOffsetAutoMax": 0,
    "titleOffsetAutoMargin": 0
  },
  "legend": {
    "orient": "left",
    "padding": 0,
    "margin": 0,
    "labelColor": "#758290",
    "labelFontSize": 12,
    "labelFont": "\"Roboto\"",
    "labelAlign": "left",
    "labelBaseline": "Bottom",
    "labelOffset": 0,
    "symbolShape": "line",
    "symbolSize": 2,
    "symbolStrokeWidth": 0,
    "titleFont": "\"Roboto\"",
    "titleFontSize": 12,
    "titleFontWeight": "medium",
    "titleColor": "#758290"
  },
  "color": {
    "rgb": [128,128,128],
    "lab": [50,0,0],
    "hcl": [0,0,50],
    "hsl": [0,0,0.5]
  },
  "range": {
    "cropColor":[
      "#3d6ac4",
      "#749be9",
      "#51d4f0",
      "#b1f3b7",
      "#24962e"
    ],
    "riskColor":[
      "#808080",
      "#FFFF99",
      "#FFE600",
      "#FF9900",
      "#FF1900",
      "#990000"
    ],
    "shapes": [
      "circle",
      "cross",
      "diamond",
      "square",
      "triangle-down",
      "triangle-up",
      "line",
      "textMark"
    ],
    "cropCatColor": [
      "#C71585",
      "#0000FF",
      "#A52A2A",
      "#FFA500",
      "#6B8E23",
      "#FFDAB9",
      "#E6E6FA",
      "#8B4513",
      "#008000",
      "#2F4F4F"
    ]
  }
}
"""

In [14]:
@register
class CodeArea(DOMWidget):
    _view_name = Unicode('CodeView').tag(sync=True)
    _view_module = Unicode('CodeArea_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)

    # Attributes
    value = Unicode('codeAreaHere', help="Code area here").tag(sync=True)
    disabled = Bool(False, help="Enable or disable user changes.").tag(sync=True)
    
    #methods
    def clear(self):
        self.value = ''

In [ ]:
%%html
<style>
.CodeMirror {
  /* Set height, width, borders, and global font properties here */
  font-family: monospace;
  height: 300px;
  color: black;
  direction: ltr;
}

/* PADDING */

.CodeMirror-lines {
  padding: 4px 0; /* Vertical padding around content */
}
.CodeMirror pre {
  padding: 0 4px; /* Horizontal padding of content */
}

.CodeMirror-scrollbar-filler, .CodeMirror-gutter-filler {
  background-color: white; /* The little square between H and V scrollbars */
}

/* GUTTER */

.CodeMirror-gutters {
  border-right: 1px solid #ddd;
  background-color: #f7f7f7;
  white-space: nowrap;
}
.CodeMirror-linenumbers {}
.CodeMirror-linenumber {
  padding: 0 3px 0 5px;
  min-width: 20px;
  text-align: right;
  color: #999;
  white-space: nowrap;
}

.CodeMirror-guttermarker { color: black; }
.CodeMirror-guttermarker-subtle { color: #999; }

/* CURSOR */

.CodeMirror-cursor {
  border-left: 1px solid black;
  border-right: none;
  width: 0;
}
/* Shown when moving in bi-directional text */
.CodeMirror div.CodeMirror-secondarycursor {
  border-left: 1px solid silver;
}
.cm-fat-cursor .CodeMirror-cursor {
  width: auto;
  border: 0 !important;
  background: #7e7;
}
.cm-fat-cursor div.CodeMirror-cursors {
  z-index: 1;
}
.cm-fat-cursor-mark {
  background-color: rgba(20, 255, 20, 0.5);
  -webkit-animation: blink 1.06s steps(1) infinite;
  -moz-animation: blink 1.06s steps(1) infinite;
  animation: blink 1.06s steps(1) infinite;
}
.cm-animate-fat-cursor {
  width: auto;
  border: 0;
  -webkit-animation: blink 1.06s steps(1) infinite;
  -moz-animation: blink 1.06s steps(1) infinite;
  animation: blink 1.06s steps(1) infinite;
  background-color: #7e7;
}
@-moz-keyframes blink {
  0% {}
  50% { background-color: transparent; }
  100% {}
}
@-webkit-keyframes blink {
  0% {}
  50% { background-color: transparent; }
  100% {}
}
@keyframes blink {
  0% {}
  50% { background-color: transparent; }
  100% {}
}

/* Can style cursor different in overwrite (non-insert) mode */
.CodeMirror-overwrite .CodeMirror-cursor {}

.cm-tab { display: inline-block; text-decoration: inherit; }

.CodeMirror-rulers {
  position: absolute;
  left: 0; right: 0; top: -50px; bottom: 0;
  overflow: hidden;
}
.CodeMirror-ruler {
  border-left: 1px solid #ccc;
  top: 0; bottom: 0;
  position: absolute;
}

/* DEFAULT THEME */

.cm-s-default .cm-header {color: blue;}
.cm-s-default .cm-quote {color: #090;}
.cm-negative {color: #d44;}
.cm-positive {color: #292;}
.cm-header, .cm-strong {font-weight: bold;}
.cm-em {font-style: italic;}
.cm-link {text-decoration: underline;}
.cm-strikethrough {text-decoration: line-through;}

.cm-s-default .cm-keyword {color: #708;}
.cm-s-default .cm-atom {color: #219;}
.cm-s-default .cm-number {color: #164;}
.cm-s-default .cm-def {color: #00f;}
.cm-s-default .cm-variable,
.cm-s-default .cm-punctuation,
.cm-s-default .cm-property,
.cm-s-default .cm-operator {}
.cm-s-default .cm-variable-2 {color: #05a;}
.cm-s-default .cm-variable-3, .cm-s-default .cm-type {color: #085;}
.cm-s-default .cm-comment {color: #a50;}
.cm-s-default .cm-string {color: #a11;}
.cm-s-default .cm-string-2 {color: #f50;}
.cm-s-default .cm-meta {color: #555;}
.cm-s-default .cm-qualifier {color: #555;}
.cm-s-default .cm-builtin {color: #30a;}
.cm-s-default .cm-bracket {color: #997;}
.cm-s-default .cm-tag {color: #170;}
.cm-s-default .cm-attribute {color: #00c;}
.cm-s-default .cm-hr {color: #999;}
.cm-s-default .cm-link {color: #00c;}

.cm-s-default .cm-error {color: #f00;}
.cm-invalidchar {color: #f00;}

.CodeMirror-composing { border-bottom: 2px solid; }

/* Default styles for common addons */

div.CodeMirror span.CodeMirror-matchingbracket {color: #0b0;}
div.CodeMirror span.CodeMirror-nonmatchingbracket {color: #a22;}
.CodeMirror-matchingtag { background: rgba(255, 150, 0, .3); }
.CodeMirror-activeline-background {background: #e8f2ff;}

/* STOP */

/* The rest of this file contains styles related to the mechanics of
   the editor. You probably shouldn't touch them. */

.CodeMirror {
  position: relative;
  overflow: hidden;
  background: white;
}

.CodeMirror-scroll {
  overflow: scroll !important; /* Things will break if this is overridden */
  /* 30px is the magic margin used to hide the element's real scrollbars */
  /* See overflow: hidden in .CodeMirror */
  margin-bottom: -30px; margin-right: -30px;
  padding-bottom: 30px;
  height: 100%;
  outline: none; /* Prevent dragging from highlighting the element */
  position: relative;
}
.CodeMirror-sizer {
  position: relative;
  border-right: 30px solid transparent;
}

/* The fake, visible scrollbars. Used to force redraw during scrolling
   before actual scrolling happens, thus preventing shaking and
   flickering artifacts. */
.CodeMirror-vscrollbar, .CodeMirror-hscrollbar, .CodeMirror-scrollbar-filler, .CodeMirror-gutter-filler {
  position: absolute;
  z-index: 6;
  display: none;
}
.CodeMirror-vscrollbar {
  right: 0; top: 0;
  overflow-x: hidden;
  overflow-y: scroll;
}
.CodeMirror-hscrollbar {
  bottom: 0; left: 0;
  overflow-y: hidden;
  overflow-x: scroll;
}
.CodeMirror-scrollbar-filler {
  right: 0; bottom: 0;
}
.CodeMirror-gutter-filler {
  left: 0; bottom: 0;
}

.CodeMirror-gutters {
  position: absolute; left: 0; top: 0;
  min-height: 100%;
  z-index: 3;
}
.CodeMirror-gutter {
  white-space: normal;
  height: 100%;
  display: inline-block;
  vertical-align: top;
  margin-bottom: -30px;
}
.CodeMirror-gutter-wrapper {
  position: absolute;
  z-index: 4;
  background: none !important;
  border: none !important;
}
.CodeMirror-gutter-background {
  position: absolute;
  top: 0; bottom: 0;
  z-index: 4;
}
.CodeMirror-gutter-elt {
  position: absolute;
  cursor: default;
  z-index: 4;
}
.CodeMirror-gutter-wrapper ::selection { background-color: transparent }
.CodeMirror-gutter-wrapper ::-moz-selection { background-color: transparent }

.CodeMirror-lines {
  cursor: text;
  min-height: 1px; /* prevents collapsing before first draw */
}
.CodeMirror pre {
  /* Reset some styles that the rest of the page might have set */
  -moz-border-radius: 0; -webkit-border-radius: 0; border-radius: 0;
  border-width: 0;
  background: transparent;
  font-family: inherit;
  font-size: inherit;
  margin: 0;
  white-space: pre;
  word-wrap: normal;
  line-height: inherit;
  color: inherit;
  z-index: 2;
  position: relative;
  overflow: visible;
  -webkit-tap-highlight-color: transparent;
  -webkit-font-variant-ligatures: contextual;
  font-variant-ligatures: contextual;
}
.CodeMirror-wrap pre {
  word-wrap: break-word;
  white-space: pre-wrap;
  word-break: normal;
}

.CodeMirror-linebackground {
  position: absolute;
  left: 0; right: 0; top: 0; bottom: 0;
  z-index: 0;
}

.CodeMirror-linewidget {
  position: relative;
  z-index: 2;
  padding: 0.1px; /* Force widget margins to stay inside of the container */
}

.CodeMirror-widget {}

.CodeMirror-rtl pre { direction: rtl; }

.CodeMirror-code {
  outline: none;
}

/* Force content-box sizing for the elements where we expect it */
.CodeMirror-scroll,
.CodeMirror-sizer,
.CodeMirror-gutter,
.CodeMirror-gutters,
.CodeMirror-linenumber {
  -moz-box-sizing: content-box;
  box-sizing: content-box;
}

.CodeMirror-measure {
  position: absolute;
  width: 100%;
  height: 0;
  overflow: hidden;
  visibility: hidden;
}

.CodeMirror-cursor {
  position: absolute;
  pointer-events: none;
}
.CodeMirror-measure pre { position: static; }

div.CodeMirror-cursors {
  visibility: hidden;
  position: relative;
  z-index: 3;
}
div.CodeMirror-dragcursors {
  visibility: visible;
}

.CodeMirror-focused div.CodeMirror-cursors {
  visibility: visible;
}

.CodeMirror-selected { background: #d9d9d9; }
.CodeMirror-focused .CodeMirror-selected { background: #d7d4f0; }
.CodeMirror-crosshair { cursor: crosshair; }
.CodeMirror-line::selection, .CodeMirror-line > span::selection, .CodeMirror-line > span > span::selection { background: #d7d4f0; }
.CodeMirror-line::-moz-selection, .CodeMirror-line > span::-moz-selection, .CodeMirror-line > span > span::-moz-selection { background: #d7d4f0; }

.cm-searching {
  background-color: #ffa;
  background-color: rgba(255, 255, 0, .4);
}

/* Used to force a border model for a node */
.cm-force-border { padding-right: .1px; }

@media print {
  /* Hide the cursor when printing */
  .CodeMirror div.CodeMirror-cursors {
    visibility: hidden;
  }
}

/* See issue #2901 */
.cm-tab-wrap-hack:after { content: ''; }

/* Help users use markselection to safely style text background */
span.CodeMirror-selectedtext { background: none; }
</style>

In [17]:
%%javascript
require.undef('CodeArea_widget');



var CodeMirror = require('codemirror/lib/codemirror');
require('codemirror/mode/javascript/javascript');
//require('codemirror/theme/monokai.css');



define('CodeArea_widget', ["@jupyter-widgets/base"], function(widgets) {
    
    var CodeModel = widgets.DOMWidgetModel.extend({
    defaults: _.extend(widgets.DOMWidgetModel.prototype.defaults(), {
        _model_name: 'DrawingModel',
        _view_name: 'DrawingView',
        _model_module: 'jupyter-drawing-pad',
        _view_module: 'jupyter-drawing-pad',
        _model_module_version: '0.1.6',
        _view_module_version: '0.1.6',
        value: 'Hello World',

    })
});
    
    var CodeView = widgets.DOMWidgetView.extend({

        // Render the view.
        render: function() {
            this.code_input = document.createElement('textarea');
            
            this.code_input.id = 'code';
            this.code_input.value = this.model.get('value');
            this.code_input.disabled = this.model.get('disabled');
            //this.code_input.setAttribute('name', 'post');
            //this.code_input.setAttribute('maxlength', 5000);
            //this.code_input.setAttribute('cols',80);
            //this.code_input.setAttribute('rows', 40);
            

            // Python -> JavaScript update
            this.model.on('change:value', this.value_changed, this);
            this.model.on('change:disabled', this.disabled_changed, this);

            // JavaScript -> Python update
            this.code_input.onchange = this.input_changed.bind(this);
            this.code_input.onchange = function() {
                console.log(this)
            }
            
            this.el.appendChild(this.code_input);
            
            //use the codemirror to build up custom textarea
            CodeMirror.fromTextArea(this.code_input, {
              mode: 'javascript',
              lineWrapping: true,
              extraKeys: {
                'Ctrl-Space': 'autocomplete'
              },
              lineNumbers: true,
              theme: 'monokai',
              cursorScrollMargin: 5
            });

        },

        value_changed: function() {
            this.code_input.value = this.model.get('value');
        },

        disabled_changed: function() {
            this.code_input.disabled = this.model.get('disabled');
        },

        input_changed: function() {
            this.model.set('value', this.code_input.value);
            console.log(this.code_input.value)
            this.model.save_changes();
        },
    });
    

    return {
        CodeModel: CodeModel,
        CodeView: CodeView
    };
});

<IPython.core.display.Javascript object>

In [18]:
codear=CodeArea(value = 'blalblablab')
codear

CodeArea(value='blalblablab')

In [12]:
codear.value

'blalblablab'

In [19]:
def patchWidget(widgetId, datasetId, body):
    return

def formatWidget2read(widgetConfig):
    object2format= [
       {'pattern':r"({{([^}}]*)(year)}})", 'sub':'2020'},
       {'pattern':r"({{([^}}]*)(water_column)}})", 'sub':'ws2028tl'},
       {'pattern':r"({{([^}}]*)(crop)}})", 'sub':'banana'},
        {'pattern':r"({{([^}}]*)(commodity)}})", 'sub':'banana'},
       {'pattern':r"({{([^}}]*)(iso)}})", 'sub':'ESP'},
        {'pattern':r"({{([^}}]*)(countryName)}})", 'sub':'Spain'}
                   ]
    
    n=json.dumps(widgetConfig)
    for paterns in object2format:
        n = re.compile(paterns['pattern']).sub(paterns['sub'], n)
    return json.loads(n)



In [20]:
wApp=widgets.Dropdown(
    options={'Resource Watch': 'rw', 'Aqueduct': 'aqueduct', 'Prep': 'prep'},
    value='aqueduct',
    description='Choose Aaplication:',
)
wsize=widgets.IntSlider(min=1, max=1e3, step=1,continuous_update=False, value=1000)

wWidgets=widgets.Dropdown(
    description='Choose widgets:',
)
box_layout = Layout(display='flex',
                    flex_flow='column',
                    height='500px',
                    width='auto')
wCodeBox= CodeArea(
    value='',
    placeholder='Type something',
    description='Widget:',
    disabled=False,
    layout=box_layout
)
wCodeBox2= widgets.Textarea(
    value='',
    placeholder='Type something',
    description='Widget Id:',
    disabled=False
)
wCodeBox3= widgets.Textarea(
    value='',
    placeholder='Type something',
    description='Dataset Id:',
    disabled=False
)
codeArea = widgets.Output()


def f(app, lsize, codeAreas, swidget_id):
    widgetsUrl="https://api.resourcewatch.org/v1/widget?app={app}&page[size]={size}".format(app=app, size=lsize)
    
    r=requests.get(widgetsUrl)
    if r.status_code == 200:
        opt = {widget['attributes']['name']: widget['id'] for widget in r.json()['data']}
        wWidgets.options = opt
        
        widgetUrl="https://api.resourcewatch.org/v1/widget/{widgetId}".format(widgetId=swidget_id)
        s=requests.get(widgetUrl)
        if s.status_code == 200:
            mywidget = s.json()['data']['attributes']['widgetConfig']
            mywidget['width']=300
            mywidget['height']=300
            if app=='aqueduct':
                mywidget['padding']="auto"
                #mywidget['viewport']=[300,300]
                display(Vega(formatWidget2read(mywidget)))
                wCodeBox.value = json.dumps(mywidget, sort_keys=True, indent=2)
                wCodeBox2.value = swidget_id
                wCodeBox3.value = s.json()['data']['attributes']['dataset']
            else:
                #wCodeBox.value = json.dumps(mywidget)
                wCodeBox2.value = swidget_id
                wCodeBox3.value = s.json()['data']['attributes']['dataset']
                display(Vega3(mywidget))
                   

out = widgets.interactive_output(f, {'app': wApp, 'lsize':wsize, 'codeAreas':wCodeBox,'swidget_id':wWidgets})
ui = widgets.VBox([widgets.HBox([wApp,wsize]),wWidgets,widgets.HBox([wCodeBox,widgets.VBox([wCodeBox3,wCodeBox2,codeArea]),out])])
#ui = widgets.VBox([widgets.HBox([wApp,wsize]),wWidgets,widgets.HBox([codeArea,widgets.VBox([wCodeBox3,wCodeBox2]),out])])
display(ui)
#out.observe(handle_slider_change, names='All')
